In [1]:
# Based on: https://www.tensorflow.org/lite/guide/python
# And then https://github.com/tensorflow/tensorflow/tree/master/tensorflow/lite/examples/python/

In [2]:
!pip3 install --index-url https://google-coral.github.io/py-repo/ tflite_runtime

Looking in indexes: https://google-coral.github.io/py-repo/
     |████████████████████████████████| 1.5 MB 14.4 MB/s 


In [3]:
# Get photo
!curl https://raw.githubusercontent.com/tensorflow/tensorflow/master/tensorflow/lite/examples/label_image/testdata/grace_hopper.bmp > /tmp/grace_hopper.bmp
# Get model
!curl https://storage.googleapis.com/download.tensorflow.org/models/mobilenet_v1_2018_02_22/mobilenet_v1_1.0_224.tgz | tar xzv -C /tmp
# Get labels
!curl https://storage.googleapis.com/download.tensorflow.org/models/mobilenet_v1_1.0_224_frozen.tgz  | tar xzv -C /tmp  mobilenet_v1_1.0_224/labels.txt

!mv /tmp/mobilenet_v1_1.0_224/labels.txt /tmp/

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  918k  100  918k    0     0  3506k      0 --:--:-- --:--:-- --:--:-- 3506k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0./
./mobilenet_v1_1.0_224.tflite
  5 89.9M    5 4904k    0     0  6582k      0  0:00:13 --:--:--  0:00:13 6573k./mobilenet_v1_1.0_224.ckpt.meta
./mobilenet_v1_1.0_224.ckpt.index
./mobilenet_v1_1.0_224.ckpt.data-00000-of-00001
 26 89.9M   26 24.0M    0     0  13.8M      0  0:00:06  0:00:01  0:00:05 13.8M./mobilenet_v1_1.0_224_info.txt
./mobilenet_v1_1.0_224_frozen.pb
100 89.9M  100 89.9M    0     0  31.5M      0  0:00:02  0:00:02 --:--:-- 31.4M
./mobilenet_v1_1.0_224_eval.pbtxt
  % Total    % Received % Xferd  Average Spee

In [4]:
model_file = "/tmp/mobilenet_v1_1.0_224.tflite"
label_file = "/tmp/labels.txt"
image_file = "/tmp/grace_hopper.bmp"
input_mean = 127.5
input_std = 127.5

In [6]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import argparse
import time

import numpy as np
from PIL import Image

import tflite_runtime.interpreter as tflite

def load_labels(filename):
  with open(filename, 'r') as f:
    return [line.strip() for line in f.readlines()]


interpreter = tflite.Interpreter(model_path=model_file)

interpreter.allocate_tensors()

input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

# check the type of the input tensor
floating_model = input_details[0]['dtype'] == np.float32

# NxHxWxC, H:1, W:2
height = input_details[0]['shape'][1]
width = input_details[0]['shape'][2]
img = Image.open(image_file).resize((width, height))

# add N dim
input_data = np.expand_dims(img, axis=0)

if floating_model:
  input_data = (np.float32(input_data) - input_mean) / input_std

interpreter.set_tensor(input_details[0]['index'], input_data)

start_time = time.time()
interpreter.invoke()
stop_time = time.time()

output_data = interpreter.get_tensor(output_details[0]['index'])
results = np.squeeze(output_data)

top_k = results.argsort()[-5:][::-1]
labels = load_labels(label_file)
for i in top_k:
  if floating_model:
    print('{:08.6f}: {}'.format(float(results[i]), labels[i]))
  else:
    print('{:08.6f}: {}'.format(float(results[i] / 255.0), labels[i]))

print('time: {:.3f}ms'.format((stop_time - start_time) * 1000))

0.919721: 653:military uniform
0.017762: 907:Windsor tie
0.007507: 668:mortarboard
0.005419: 466:bulletproof vest
0.003828: 458:bow tie, bow-tie, bowtie
time: 113.866ms


In [8]:
# Save the model
import tensorflow as tf

# Convert the model to the TensorFlow Lite format without quantization
converter = tf.lite.TFLiteConverter.from_keras_model(model_2)
tflite_model = converter.convert()

# Save the model to disk
open("sine_model.tflite", "wb").write(tflite_model)

# Convert the model to the TensorFlow Lite format with quantization
converter = tf.lite.TFLiteConverter.from_keras_model(model_2)
# Indicate that we want to perform the default optimizations,
# which include quantization
converter.optimizations = [tf.lite.Optimize.DEFAULT]
# Define a generator function that provides our test data's x values
# as a representative dataset, and tell the converter to use it
def representative_dataset_generator():
  for value in x_test:
    # Each scalar value must be inside of a 2D array that is wrapped in a list
    yield [np.array(value, dtype=np.float32, ndmin=2)]
converter.representative_dataset = representative_dataset_generator
# Convert the model
tflite_model = converter.convert()

# Save the model to disk
open("sine_model_quantized.tflite", "wb").write(tflite_model)

NameError: ignored

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=b5d4c7db-26fd-4ded-b22d-b8b006278bac' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>